In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Function to read data from CSV file
def read_data(pathCSV):
    filenames = []
    labels = []
    
    df = pd.read_csv(pathCSV)
    
    for index, row in df.iterrows():
        filename = row['filename']
        class_name = row['class']
        
        filenames.append(filename)
        labels.append(class_name)
        
    return filenames, labels

# Directories for train, valid, test data
train_data_dir = '/kaggle/input/learntools/train/'
validation_data_dir = '/kaggle/input/learntools/valid/'
test_data_dir = '/kaggle/input/learntools/test/'

# Read data from train, valid, and test sets
train_filenames, train_labels = read_data(os.path.join(train_data_dir, "_annotations.csv"))
test_filenames, test_labels = read_data(os.path.join(test_data_dir, "_annotations.csv"))
validation_filenames, validation_labels = read_data(os.path.join(validation_data_dir, "_annotations.csv"))


# Xử lý dữ liệu hình ảnh từ tập train
train_images = []
train_labels_list = []
for filename, label in zip(train_filenames, train_labels):
    image = cv2.imread(os.path.join(train_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    train_images.append(image)
    train_labels_list.append(label)

train_images = np.array(train_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
train_labels = pd.get_dummies(train_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

# Xử lý dữ liệu hình ảnh từ tập valid
valid_images = []
valid_labels_list = []
for filename, label in zip(validation_filenames, validation_labels):
    image = cv2.imread(os.path.join(validation_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    valid_images.append(image)
    valid_labels_list.append(label)

valid_images = np.array(valid_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
valid_labels = pd.get_dummies(valid_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

# Xử lý dữ liệu hình ảnh từ tập test
test_images = []
test_labels_list = []
for filename, label in zip(test_filenames, test_labels):
    image = cv2.imread(os.path.join(test_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    test_images.append(image)
    test_labels_list.append(label)

test_images = np.array(test_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
test_labels = pd.get_dummies(test_labels_list)  # Chuyển đổi nhãn sang one-hot encoding

In [ ]:
# Import các thư viện cần thiết
import os
import pandas as pd
import cv2
import numpy as np

# Đọc dữ liệu từ các tệp CSV
test_df = pd.read_csv("/kaggle/input/learntools/test/_annotations.csv")
train_df = pd.read_csv("/kaggle/input/learntools/train/_annotations.csv")
valid_df = pd.read_csv("/kaggle/input/learntools/valid/_annotations.csv")

# Hàm đọc ảnh và xử lý bounding box
def process_data(dataframe, data_dir):
    images = []
    labels = []
    for index, row in dataframe.iterrows():
        filename = row['filename']
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)  # Đọc ảnh
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Chuyển đổi màu sắc từ BGR sang RGB
            images.append(image)
            
            # Chuẩn hóa bounding box
            xmin = row['xmin']
            ymin = row['ymin']
            xmax = row['xmax']
            ymax = row['ymax']
            width = row['width']  # Chiều rộng ảnh
            height = row['height']  # Chiều cao ảnh
            # Chuẩn hóa tọa độ bounding box
            xmin_norm = xmin / width
            ymin_norm = ymin / height
            xmax_norm = xmax / width
            ymax_norm = ymax / height
            
            labels.append([xmin_norm, ymin_norm, xmax_norm, ymax_norm])
    
    return np.array(images), np.array(labels)

# Xử lý dữ liệu cho tập train, validation và test
train_images_vgg16, train_labels_vgg16 = process_data(train_df, "/kaggle/input/learntools/train/")
valid_images_vgg16, valid_labels_vgg16 = process_data(valid_df, "/kaggle/input/learntools/valid/")
test_images_vgg16, test_labels_vgg16 = process_data(test_df, "/kaggle/input/learntools/test/")

In [ ]:
from tensorflow.keras.models import load_model

# Đường dẫn tới file model trên Kaggle
model_path = "/kaggle/input/models-classification/classification_CNN_model.h5"

# Load model từ file
cnn_model = load_model(model_path)

In [ ]:
# Đánh giá mô hình trên tập test
test_loss_improved_cnn, test_accuracy_improved_cnn = cnn_model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss_improved_cnn)
print("Test Accuracy:", test_accuracy_improved_cnn)

In [ ]:
import shutil

# Đường dẫn tới model trong thư mục chỉ đọc
source_model_path = "/kaggle/input/bbox-best-model/vgg16_model_v2.keras"

# Đường dẫn đến thư mục có thể ghi được
destination_model_path = "/kaggle/working/vgg16_model_v2.keras"

# Sao chép model từ thư mục chỉ đọc sang thư mục có thể ghi được
shutil.copyfile(source_model_path, destination_model_path)

# Load model từ thư mục có thể ghi được
vgg16_model = load_model(destination_model_path)


In [ ]:
# Đánh giá mô hình trên tập test
test_loss_vgg16, test_accuracy_vgg16 = vgg16_model.evaluate(test_images_vgg16, test_labels_vgg16)
print("Test Loss:", test_loss_vgg16)
print("Test Accuracy:", test_accuracy_vgg16)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Dense

# Tạo input cho mỗi mô hình
cnn_input = Input(shape=(224, 224, 3))
vgg16_input = Input(shape=(224, 224, 3))

# Lấy output từ mô hình CNN
cnn_output = cnn_model(cnn_input)

# Lấy output từ mô hình VGG16
vgg16_output = vgg16_model(vgg16_input)

# Kết hợp outputs từ cả hai mô hình
combined_output = concatenate([cnn_output, vgg16_output])

# Thêm các lớp fully connected và output
combined_output = Dense(512, activation='relu')(combined_output)
combined_output = Dense(5, activation='softmax')(combined_output)  # Số lớp phân loại

# Tạo mô hình đa nhánh
combined_model = Model(inputs=[cnn_input, vgg16_input], outputs=combined_output)

# Compile mô hình
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Hiển thị tóm tắt của mô hình
combined_model.summary()

# Huấn luyện mô hình
history = combined_model.fit([train_images, train_images_vgg16], train_labels, epochs=10, batch_size=32, validation_data=([valid_images, valid_images_vgg16], valid_labels))

# Đánh giá mô hình trên tập test
test_loss, test_accuracy = combined_model.evaluate([test_images, test_images_vgg16], test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import matplotlib.pyplot as plt

# Biểu đồ loss và accuracy
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Dense, Conv2D, MaxPooling2D, Flatten

# Tạo input cho mỗi mô hình
cnn_input = Input(shape=(224, 224, 3))
vgg16_input = Input(shape=(224, 224, 3))

# Tạo mô hình CNN
cnn_output = Conv2D(32, (3, 3), activation='relu')(cnn_input)
cnn_output = MaxPooling2D((2, 2))(cnn_output)
cnn_output = Conv2D(64, (3, 3), activation='relu')(cnn_output)
cnn_output = MaxPooling2D((2, 2))(cnn_output)
cnn_output = Conv2D(128, (3, 3), activation='relu')(cnn_output)
cnn_output = MaxPooling2D((2, 2))(cnn_output)
cnn_output = Flatten()(cnn_output)

# Tạo mô hình VGG16
vgg16_output = vgg16_model(vgg16_input)

# Kết hợp outputs từ cả hai mô hình
combined_output = concatenate([cnn_output, vgg16_output])

# Thêm các lớp fully connected và output
combined_output = Dense(512, activation='relu')(combined_output)
combined_output = Dense(5, activation='softmax')(combined_output)  # Số lớp phân loại

# Tạo mô hình đa nhánh
combined_model = Model(inputs=[cnn_input, vgg16_input], outputs=combined_output)

# Compile mô hình
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Hiển thị tóm tắt của mô hình
combined_model.summary()

In [ ]:
# Huấn luyện mô hình
history = combined_model.fit([train_images, train_images_vgg16], train_labels, epochs=50, batch_size=32,
                             validation_data=([valid_images, valid_images_vgg16], valid_labels))

# Đánh giá mô hình trên tập test
test_loss, test_accuracy = combined_model.evaluate([test_images, test_images_vgg16], test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Lưu mô hình vào file HDF5
combined_model.save("/kaggle/working/combined_model.keras")

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Hàm để hiển thị ảnh với bounding box và nhãn
def visualize_image(image, bbox, label, predicted_labels):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if len(bbox.shape) == 1:  # Nếu chỉ có một bounding box
        bbox = np.expand_dims(bbox, axis=0)  # Mở rộng kích thước của bbox
    for i in range(len(bbox)):
        xmin, ymin, xmax, ymax = bbox[i] * [image.shape[1], image.shape[0], image.shape[1], image.shape[0]]
        xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        
        # Hiển thị predicted labels trong bounding box
        text = f"{predicted_labels[i]}"
        cv2.putText(image, text, (xmin + 20, ymin + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
    plt.imshow(image)
    plt.axis('off')
    plt.show()

    
# Lấy ảnh từ tập test
image_index = 9  # Chọn ảnh khác từ tập test
image_path = os.path.join(test_data_dir, test_filenames[image_index])
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Đưa ảnh vào mô hình để dự đoán
predicted_labels = combined_model.predict([test_images[image_index:image_index+1], test_images_vgg16[image_index:image_index+1]])

# Lấy nhãn của đối tượng
labels = ['calculator', 'compa', 'pen', 'pencil', 'ruler']
predicted_label_indices = np.argmax(predicted_labels, axis=1)
predicted_label_names = [labels[i] for i in predicted_label_indices]

# Lấy bounding box từ output của mô hình VGG16
bbox = test_labels_vgg16[image_index]

# Hiển thị ảnh với bounding box và nhãn
visualize_image(image, bbox, labels, predicted_label_names)


In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Function to read data from CSV file
def read_data(pathCSV):
    filenames = []
    labels = []
    
    df = pd.read_csv(pathCSV)
    
    for index, row in df.iterrows():
        filename = row['filename']
        class_name = row['class']
        
        filenames.append(filename)
        labels.append(class_name)
        
    return filenames, labels

test_data_dir = '/kaggle/input/nh-thc-t'

# Xử lý dữ liệu hình ảnh từ tập test
test_images = []
test_labels_list = []
for filename, label in zip(test_filenames, test_labels):
    image = cv2.imread(os.path.join(test_data_dir, filename))
    image = cv2.resize(image, (224, 224))  # Resize hình ảnh về kích thước 224x224
    test_images.append(image)
    test_labels_list.append(label)

test_images = np.array(test_images) / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]


In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

test_data_dir = '/kaggle/input/nh-thc-t'

# Load ảnh từ thư mục test_data_dir
test_filenames = os.listdir(test_data_dir)
test_images = []

for filename in test_filenames:
    image_path = os.path.join(test_data_dir, filename)
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize ảnh về kích thước 224x224
    image = image / 255.0  # Chuẩn hóa giá trị pixel về khoảng [0, 1]
    test_images.append(image)

test_images = np.array(test_images)

# Hàm để hiển thị ảnh với bounding box và nhãn
def visualize_image(image, bbox, label, predicted_labels):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if len(bbox.shape) == 1:  # Nếu chỉ có một bounding box
        bbox = np.expand_dims(bbox, axis=0)  # Mở rộng kích thước của bbox
    for i in range(len(bbox)):
        xmin, ymin, xmax, ymax = bbox[i] * [image.shape[1], image.shape[0], image.shape[1], image.shape[0]]
        xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        
        # Hiển thị predicted labels trong bounding box
        text = f"{predicted_labels[i]}"
        cv2.putText(image, text, (xmin + 20, ymin + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Lấy ảnh từ tập test
image_index = 2 # Chọn ảnh khác từ tập test
image_path = os.path.join(test_data_dir, test_filenames[image_index])
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Đưa ảnh vào mô hình để dự đoán
predicted_labels = combined_model.predict([test_images[image_index:image_index+1], test_images_vgg16[image_index:image_index+1]])

# Lấy nhãn của đối tượng
labels = ['calculator', 'compa', 'pen', 'pencil', 'ruler']
predicted_label_indices = np.argmax(predicted_labels, axis=1)
predicted_label_names = [labels[i] for i in predicted_label_indices]

# Lấy bounding box từ output của mô hình VGG16
bbox = test_labels_vgg16[image_index]

# Hiển thị ảnh với bounding box và nhãn
visualize_image(image, bbox, labels, predicted_label_names)
3

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

# Hàm để hiển thị ảnh với bounding box và nhãn
def visualize_image(image, bbox, label, predicted_labels):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if len(bbox.shape) == 1:  # Nếu chỉ có một bounding box
        bbox = np.expand_dims(bbox, axis=0)  # Mở rộng kích thước của bbox
    for i in range(len(bbox)):
        xmin, ymin, xmax, ymax = bbox[i] * [image.shape[1], image.shape[0], image.shape[1], image.shape[0]]
        xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        
        # Hiển thị predicted labels trong bounding box
        text = f"{predicted_labels[i]}"
        cv2.putText(image, text, (xmin + 20, ymin + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Lấy bounding box từ output của mô hình kết hợp
combined_output = combined_model.predict([np.expand_dims(new_image, axis=0), np.expand_dims(new_image, axis=0)])
predicted_bbox = combined_output[:, -4:]  # Giả sử bounding box là 4 phần tử cuối cùng của output

# Lấy ảnh từ đường dẫn hoặc bất kỳ nguồn nào khác
image_path = '/kaggle/input/nh-thc-t/pngtree-ballpoint-pen-photography-image_889170.jpg'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Hiển thị ảnh với bounding box và nhãn
visualize_image(image, predicted_bbox, labels, predicted_label_names)
